In [6]:
# Setup the Jupyter version of Dash
from dash import Dash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
import base64
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Module6Milestone import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

USER = "aacUser"
PASS = "SNHU1234"

shelter = AnimalShelter(USER, PASS)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = Dash('SimpleExample')

# Check here Image
image_name = 'GraziosoSalvareLogo.png'
# encoded_image = base64.b64encode(open(image_name, 'rb').read())

app.layout = html.Div([
    html.A([
        # html.Img(id='Logo',src='data:image/png;base64,{}'.format(encoded_image.decode())),
    ], href="snhu.edu"),
    html.Center(html.B(html.H1('Samuel Walters CS-340 Dashboard'))),
    html.Hr(),
    # Check here Radio buttons
    html.Div(
        # Add in radio buttons that will contain the different options
        dcc.RadioItems(
            id='filter_by_selection',
            options=[
                {'label': 'Water', 'value' : 'water_rescue'},
                {'label': 'Mountain / Wilderness', 'value' : 'mountain_wilderness'},
                {'label': 'Disaster / Tracking', 'value' : 'disaster_tracking'},
                {'label': 'Reset', 'value' : 'reset'},
            ],
            value = 'reset',
            labelStyle={'display': 'inline-flex'}
        )
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter_by_selection', 'value')])
def update_dashboard(filter_by_selection):
    # Will change based on which radio button. Used if statements as Switch is not in Python 3.9
    if filter_by_selection == 'water_rescue':
            df = pd.DataFrame(list(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            })))
    elif filter_by_selection == 'mountain_wilderness':
            df = pd.DataFrame(list(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepherd","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            })))
    elif filter_by_selection == 'disaster_tracking':
            df = pd.DataFrame(list(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepherd","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            })))
        #resets the search to nothing to allow all results to be displayed
    elif filter_by_selection == 'reset':
        df = pd.DataFrame.from_records(shelter.read({}))
        
    df.drop(columns=['_id'],inplace=True) # Drop _id column or will not display
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
    return (data,columns)

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    #creates the values needed for the names and breeds.
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #creates a pie chart based on the data above
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500   
            )
        )
    ]
@app.callback(
    Output('map-id', "children"),    
    [Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData):
    #create the views
    if not viewData: #Default view will be the center of Austin Texas
        markerArray = (30.75,-97.48) #default marker at Austin City center
        toolTip = "Austin Texas"
        popUpHeading="Austin Texas"
        popUpParagraph="Austin Texas"

        
    else: # Update the marker view
        dff = pd.DataFrame(df.iloc[viewData]) #convert the datatable to pandas dataframe
        coordLat = float(dff['location_lat'].to_string().split()[1]) # Get the latitude
        coordLong = float(dff['location_long'].to_string().split()[1]) # Get the longitude
        markerArray = (coordLat, coordLong) # Combine them
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']

    return [dl.Map(style={'width': '600px', 'height': '300px'}, center=[30.75,-97.48],
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
            ])
        ])
    ])
]


app.run_server(debug=True)    # Uncomment this line to enable debugging.
# app.run_server()                # Comment this line if enabling debugging

Connection Successful


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[6], line 182, in update_graphs(viewData=None)
    180 dff = pd.DataFrame.from_dict(viewData)
    181 #creates the values needed for the names and breeds.
--> 182 names = dff['breed'].value_counts().keys().tolist()
        dff = Empty DataFrame
Columns: []
Index: []
    183 values = dff['breed'].value_counts().tolist()
    184 #creates a pie chart based on the data above

File c:\Users\Samuel\source\repos\SNHU_CS_340\.venv\Lib\site-packages\pandas\core\frame.py:4102, in DataFrame.__getitem__(
    self=Empty DataFrame
Columns: []
Index: [],
    key='breed'
)
   4100 if self.columns.nlevels > 1:
   4101     return self._getitem_multilevel(key)
-> 4102 indexer = self.columns.get_loc(key)
        key = 'breed'
        self = Empty DataFrame
Columns: []
Index: []
   4103 if is_integer(indexer):
   4104     indexer = [indexer]

File c: